In [12]:
import pandas as pd
import numpy as np
import plotly.express as px

In [13]:
order_products = pd.read_csv("order_products__train.csv")
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")
aisles = pd.read_csv("aisles.csv")
departments = pd.read_csv("departments.csv")

In [15]:
# Orders per user

orders_per_user = orders.groupby("user_id", as_index=False).order_number.count()
orders_per_user.rename(columns={"order_number":"order_count"}, inplace=True)

order_count_distribution = orders_per_user.groupby("order_count", as_index=False).user_id.count()
order_count_distribution.rename(columns={"user_id":"user_count"}, inplace=True)

fig = px.bar(order_count_distribution, x="order_count", y="user_count")
fig.show()